# abopt

## 0. Import

In [ ]:
import os, sys

import numpy as np
import pandas as pd

import abopt

## 1. Estimator

In [ ]:
## Read in the data
datapath = "../data/estimator/"
filename = "NeutSeqData_VH3-53_66_aligned.csv"
df = pd.read_csv(os.path.join(datapath, filename), sep=",", header=0)

In [ ]:
## Inspect dataframe
df.head()

In [ ]:
## Run estimator to and receive dictionary back with satlasso object, 
## coefficients, and coefficients mapped back to original sequences of each antibody in training data
output_dict = abopt.estimator.fit_estimator(df = df, y_colname = "IC50_ngml", \
                           sequence_colname = "sequence", id_colname = "antibody_id", \
                           lambda1 = 1., lambda2 = 10., lambda3 = 10., \
                           heavy_chain_colname = "heavy_chain_aligned", \
                           light_chain_colname = "light_chain_aligned", \
                           saturation = "mode", map_back = True, cv = 0)

In [ ]:
## Inspect output
output_dict.keys()

In [ ]:
## Prepare coefficients dataframes for output (only for conforming to manuscript)
coefficients = output_dict["Coefficients"]
coefficients.replace(mapper={"Coefficients": "coefficients"}, axis=1, inplace=True)

mapped_coefficients = output_dict["Mapped Coefficients"]
mapped_coefficients.replace(mapper={"Coefficient": "coefficient", \
                                    "Position": "location", \
                                    "Chain": "chain", \
                                    "AA": "aa", \
                                    "WT": "wt"}, \
                            axis=1, inplace=True)

In [ ]:
## Output coefficients dataframes
outpath = "../output/estimator/"
coefficient_filename = filename[:-4] + "_coefficients.csv"
mapped_coefficient_filename = filename[:-4] + "_mapped_coefficients.csv"

coefficients.to_csv(coefficient_filename, sep=",", header=True, index=False)
mapped_coefficients.to_csv(mapped_coefficient_filename, sep=",", header=True, index=True)

## 2. Fitness

In [ ]:
## Filenames for virus and antibody fitness landscapes 
antibody_fitness = '../output/merge/rbd_ab_fitness_opt.csv'
virus_fitness = '../output/merge/rbd_ace2_fitness.csv'
antibody_metadata = '../data/meta/antibody_list.tsv'

In [ ]:
## Create an antibody fitness landscape from existing fitness data 
fitdata = abopt.fitness.Fitness(antibody_metadata, antibody_fitness, virus_fitness, antibody_list)

In [ ]:
## Compare antibodies across classes 
def calculate_antibody_class_differences(fitdata, group_1, group_2, group_3, group_4): 

    test12 = fitdata.compare_antibody_groups(antibody_group_1 = group_1, antibody_group_2 = group_2)
    test13 = fitdata.compare_antibody_groups(antibody_group_1 = group_1, antibody_group_2 = group_3)
    test14 = fitdata.compare_antibody_groups(antibody_group_1 = group_1, antibody_group_2 = group_4)
    test23 = fitdata.compare_antibody_groups(antibody_group_1 = group_2, antibody_group_2 = group_3)
    test24 = fitdata.compare_antibody_groups(antibody_group_1 = group_2, antibody_group_2 = group_4)
    test34 = fitdata.compare_antibody_groups(antibody_group_1 = group_3, antibody_group_2 = group_4)


    sig = test12.significant | test13.significant | test14.significant | test23.significant | test24.significant | test34.significant
    test= pd.concat([test12.fold_change, test13.fold_change, test14.fold_change, test23.fold_change, test24.fold_change, test34.fold_change], axis=1)

    test['fc'] = test.max(axis=1).values
    test['significant'] = sig
    test['location'] =test12.location
    
    c = test.significant & ((test.fc >=1.25) | (test.fc <= 0.75))
    locations = list(test.loc[c].location.values)
    
    return locations 

In [ ]:
## Get the 4 classes of antibodies 
group_1 = fmt.get_antibodies(class_number='1')
group_2 = fmt.get_antibodies(class_number='2')
group_3 = fmt.get_antibodies(class_number='3')
group_4 = fmt.get_antibodies(class_number='Unknown')

locations = calculate_antibody_class_differences(fitdata, group_1, group_2, group_3, group_4)
antibodies = group_1 +group_2 + group_3 + group_4 

In [ ]:
## Case study for C105
fitdata.remove_antibodies([ab for ab in fitdata.antibodies if ab != 'C105'])

pdb_name = fitdata.pdb('C105')

In [ ]:
## Repair antibody/viral receptor original structure, outputs to output/repair 
fitdata.repair([pdb_name])

In [ ]:
## Remove virus and antibody from WT structure and repair these structures
fitdata.remove([pdb_name], chain_type= 'antibody', property='repair')
fitdata.remove([pdb_name], chain_type= 'virus', property='repair')

pdb_list = [pdb_name + '_Repair', pdb_name + '_Repair_less_ab']

In [ ]:
## Repair these structures 
fitdata.repair(pdb_list, property='remove')

In [ ]:
## Mutational scanning of C105 repaired pdbs

## Construct list of locations to scan '

location_file = '../../data/location/SARS_CoV_2_RBD_locations.csv'

posscan = fitdata.construct_position_scan_string (pdb_name='C105', location_file=location_file, chain = None, filter= [472,501])
pdb_list = [pdb_name + '_Repair', pdb_name + '_Repair_less_ab']

fitdata.scan (pdb_list=pdb_list, property='repair', scan_type='location', scan_values=posscan, scan_molecule='virus')

In [ ]:
## Constrain estimator to very large negative of positive coefficients
cutoff = 1e-8
antibody = 'C105'
constrained_coefficients = fitdata.constrain(data_type ='estimator', data_file=mapped_coefficient_filename, antibody=antibody, cutoff = [-cutoff, cutoff], top=1000)

In [ ]:
## Get pdb sequence locations on antibody that are less than 98
constrained_coefficients = constrained_coefficients.loc[constrained_coefficients.pdb_location < '98']

In [ ]:
## Compare C105 and C105 optimized
antibodies = ['C105', 'C105_TH28I_YH58F']
test = fitdata.compare_antibodies(antibody_1 = antibodies[0], antibody_2 = antibodies[1])
locations = list(test.loc[test.significant].location.values)